In [14]:
import pandas as pd
import os

In [15]:
for file in os.listdir("."):
    if file.endswith("csv"):
        df = pd.read_csv(file)
        print(file.strip("mmlu_pro_").strip("_eval_all_None.csv").strip())
        print("Missing Answer:", df["extracted_answer"].isna().sum())
        print("Missing verbal_numerical_confidence:", df["verbal_numerical_confidence"].isna().sum())
        print("Missing logit_perplexity_confidence:", df["logit_perplexity_confidence"].isna().sum())
        print("Missing verbal_linguistic_confidence:", df["verbal_linguistic_confidence"].isna().sum())
        print()


Meta-Llama-3.1-70B-Instruct-Turb
Missing Answer: 139
Missing verbal_numerical_confidence: 147
Missing logit_perplexity_confidence: 0
Missing verbal_linguistic_confidence: 0

Meta-Llama-3.1-8B-Instruct-Turb
Missing Answer: 605
Missing verbal_numerical_confidence: 614
Missing logit_perplexity_confidence: 0
Missing verbal_linguistic_confidence: 0

Llama-4-Scout-17B-16E-Instruct
Missing Answer: 24
Missing verbal_numerical_confidence: 29
Missing logit_perplexity_confidence: 0
Missing verbal_linguistic_confidence: 0

gpt-4.1-mini
Missing Answer: 183
Missing verbal_numerical_confidence: 174
Missing logit_perplexity_confidence: 0
Missing verbal_linguistic_confidence: 0

Meta-Llama-3.1-405B-Instruct-Turb
Missing Answer: 114
Missing verbal_numerical_confidence: 111
Missing logit_perplexity_confidence: 0
Missing verbal_linguistic_confidence: 0

Llama-3.2-3B-Instruct-Turb
Missing Answer: 102
Missing verbal_numerical_confidence: 104
Missing logit_perplexity_confidence: 0
Missing verbal_linguistic_c